In [2]:
## importing required libraries
import os
import shutil
import random

!pip install ultralytics
!pip install tqdm --upgrade
from tqdm.notebook import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.7/611.7 kB 12.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
train_path_img = "./yolo_data/images/train/"
train_path_label = "./yolo_data/labels/train/"
val_path_img = "./yolo_data/images/val/"
val_path_label = "./yolo_data/labels/val/"
test_path = "./yolo_data/test"

In [6]:
'''
Split the dataset into train and test and creates the train.txt and test.tx with
the respective path of the images in each folder
'''

def train_test_split(path,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images


    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)


    ### ----------- copying images to train folder
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')



    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')

        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")

        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")



    ### copytin images to validation folder
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue
      # print("running")
      shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")

    print("------ TASK COMPLETED -------")

## spliting the data into train-test and creating train.txt and test.txt files
# train_test_split('/content/drive/MyDrive/custom_notebooks/yolo_data/')

### for label_tag
train_test_split('/content/drive/MyDrive/football_with_yolov8/data/') ### without negative images
# train_test_split('./data/','./negative_images/') ### if you want to feed negative images

------ PROCESS STARTED -------
--- This folder has a total number of 66 images---


  0%|          | 0/53 [00:00<?, ?it/s]

------ Training data created with 80% split 53 images -------


  0%|          | 0/13 [00:00<?, ?it/s]

------ Testing data created with a total of 13 images ----------
------ TASK COMPLETED -------


In [7]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.120 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 24.2/78.2 GB disk)


In [11]:
!yolo task=detect mode=train model=yolov8s.pt data=/content/drive/MyDrive/football_with_yolov8/dataset.yaml epochs=10 imgsz=640 batch=8 project=/content/drive/MyDrive/football_with_yolov8/training_results name=football

Ultralytics YOLOv8.0.120 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/football_with_yolov8/dataset.yaml, epochs=10, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/football_with_yolov8/training_results, name=football, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_mask

In [14]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/football_with_yolov8/training_results/football2/weights/best.pt conf=0.55 source=/content/drive/MyDrive/football_with_yolov8/test_images

Ultralytics YOLOv8.0.120 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients

image 1/2 /content/drive/MyDrive/football_with_yolov8/test_images/train1.jpg: 448x640 17 players, 58.3ms
image 2/2 /content/drive/MyDrive/football_with_yolov8/test_images/train2.jpg: 448x640 1 player, 12.9ms
Speed: 2.7ms preprocess, 35.6ms inference, 39.6ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict


In [16]:
!cp -r /content/runs/detect/predict /content/drive/MyDrive/football_with_yolov8/output

In [17]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/football_with_yolov8/training_results/football2/weights/best.pt conf=0.77 source=/content/drive/MyDrive/football_with_yolov8/video

Ultralytics YOLOv8.0.120 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients

video 1/1 (1/501) /content/drive/MyDrive/football_with_yolov8/video/pexels-tima-miroshnichenko-6078265-3840x2160-25fps.mp4: 384x640 6 players, 84.4ms
video 1/1 (2/501) /content/drive/MyDrive/football_with_yolov8/video/pexels-tima-miroshnichenko-6078265-3840x2160-25fps.mp4: 384x640 5 players, 11.6ms
video 1/1 (3/501) /content/drive/MyDrive/football_with_yolov8/video/pexels-tima-miroshnichenko-6078265-3840x2160-25fps.mp4: 384x640 3 players, 11.1ms
video 1/1 (4/501) /content/drive/MyDrive/football_with_yolov8/video/pexels-tima-miroshnichenko-6078265-3840x2160-25fps.mp4: 384x640 2 players, 11.2ms
video 1/1 (5/501) /content/drive/MyDrive/football_with_yolov8/video/pexels-tima-miroshnichenko-6078265-3840x2160-25fps.mp4: 384x640 3 players, 11.1ms
video 1/1 (6/501) /content/drive/MyDrive/football_with_yolov8/video/pexels-tima-miroshnichenk

In [18]:
!cp -r /content/runs/detect/predict2 /content/drive/MyDrive/football_with_yolov8/output